In [1]:
#Imports
from pyspark import SparkContext #Library
sc = SparkContext("local[*]", "temp") #Start a spart context

from pyspark.ml.feature import Word2Vec
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

import xml.etree.ElementTree as ET



In [2]:
posts_lines = sc.textFile("file:///home/vagrant/miniprojects/spark/full/allPosts/*.gz") #create an RDD from file
print posts_lines
posts_lines.count()


MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:-2


47946654

In [3]:
def parsePost(line):
    if '<row'in line:
        try:
            root = ET.fromstring(line.encode('utf8').strip())
        except:
            pass
            return ("Empty")
            
        if root != '':
            if "Tags" in root.attrib:
                    try:
                        data = root.attrib["Tags"]
                    except:
                        pass
                        return ("Empty")
                    data = data.replace("><", " ").replace("<", "").replace(">", "")
                    return data
            else:
                return("Empty")
        else:
            return("Empty")
    else:
        return("Empty")
            
        

In [4]:

content = posts_lines.map(parsePost).filter(lambda x: x!= 'Empty')\
            .map(lambda line: (line.split(" "), 1)).toDF(['text', 'score'])

print content
content.count()
print content.take(20)

DataFrame[text: array<string>, score: bigint]
[Row(text=[u'optimization', u'cuda'], score=1), Row(text=[u'java', u'android', u'linux', u'networking', u'sshj'], score=1), Row(text=[u'python', u'python-3.x', u'dictionary'], score=1), Row(text=[u'android'], score=1), Row(text=[u'solr', u'elasticsearch', u'morelikethis'], score=1), Row(text=[u'android', u'android-intent', u'call'], score=1), Row(text=[u'ruby', u'dictionary'], score=1), Row(text=[u'java', u'android', u'class', u'bluetooth', u'android-bluetooth'], score=1), Row(text=[u'jsp', u'error-handling', u'struts-1.x'], score=1), Row(text=[u'bash', u'arguments'], score=1), Row(text=[u'codeigniter'], score=1), Row(text=[u'c++', u'class', u'templates'], score=1), Row(text=[u'c++', u'cmake'], score=1), Row(text=[u'actionscript-3', u'flashdevelop'], score=1), Row(text=[u'php', u'arrays', u'for-loop'], score=1), Row(text=[u'javascript', u'jquery'], score=1), Row(text=[u'php', u'mysql', u'arrays'], score=1), Row(text=[u'nexus', u'jersey-clie

In [5]:
w2v = Word2Vec(inputCol="text",vectorSize=100, seed=42L) 
model = w2v.fit(content)


In [6]:
result = model.findSynonyms('ggplot2', 40).rdd.take(40)
print result


[Row(word=u'lattice', similarity=2.500966616566001), Row(word=u'shiny', similarity=2.329235087188001), Row(word=u'data.frame', similarity=2.3228756021521852), Row(word=u'r-grid', similarity=2.316870014786195), Row(word=u'lm', similarity=2.3096607665949684), Row(word=u'plyr', similarity=2.3075690277968426), Row(word=u'boxplot', similarity=2.3047255614301383), Row(word=u'data.table', similarity=2.2902269345825337), Row(word=u'na', similarity=2.2860795623681476), Row(word=u'levelplot', similarity=2.278655040779817), Row(word=u'geom-bar', similarity=2.2679633397888206), Row(word=u'quantmod', similarity=2.249092516045453), Row(word=u'reshape', similarity=2.248116894836326), Row(word=u'ggvis', similarity=2.2455025888019824), Row(word=u'r-factor', similarity=2.244787382164097), Row(word=u'facet-wrap', similarity=2.2323297589284676), Row(word=u'vegan', similarity=2.2240827623302373), Row(word=u'dplyr', similarity=2.220983793081478), Row(word=u'lme4', similarity=2.2201058228506656), Row(word=u'

In [7]:

answer=[]
for i in range(0,25):
    answer.append((str(result[i][0]), result[i][1]))
print answer

[('lattice', 2.500966616566001), ('shiny', 2.329235087188001), ('data.frame', 2.3228756021521852), ('r-grid', 2.316870014786195), ('lm', 2.3096607665949684), ('plyr', 2.3075690277968426), ('boxplot', 2.3047255614301383), ('data.table', 2.2902269345825337), ('na', 2.2860795623681476), ('levelplot', 2.278655040779817), ('geom-bar', 2.2679633397888206), ('quantmod', 2.249092516045453), ('reshape', 2.248116894836326), ('ggvis', 2.2455025888019824), ('r-factor', 2.244787382164097), ('facet-wrap', 2.2323297589284676), ('vegan', 2.2240827623302373), ('dplyr', 2.220983793081478), ('lme4', 2.2201058228506656), ('do.call', 2.219662937376575), ('plotmath', 2.2165199201319497), ('zoo', 2.213539713024378), ('rgl', 2.212606172200445), ('gtable', 2.2107515807481612), ('read.table', 2.2106231837194112)]
